In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np #대수학
import pandas as pd #전처리

import seaborn as sns #시각화
from matplotlib.patches import Patch
from matplotlib import pyplot as plt

from tensorflow.keras.models import load_model


plt.rcParams.update({'figure.max_open_warning': 0})
plt.style.use('fivethirtyeight')
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

import warnings
%matplotlib inline

import configparser
from sklearn.preprocessing import MinMaxScaler

import re
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import datetime as dt

import os # 경로

import numpy as np
import sys
import random 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import SimpleRNN, LSTM, Bidirectional

import configparser
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import SimpleRNN
from keras.layers import Dropout
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

### covid 포함

In [4]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gdp_re/training_data_0524.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data.tail()

,Y,Y1,Y2,Y3,YY1,YY2,YY3,MONTH,YEAR,QRT,...,X16_L0,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3
72,-1.3,-3.2,2.2,1.1,['-1.3' '-3.2'],['-1.3' '-3.2' '2.2'],['-1.3' '-3.2' '2.2' '1.1'],1,2020,2020Q1,...,2119.01,2197.67,2087.96,2083.48,109.65,106.59,110.77,120.52,112.14,115.69
73,-3.2,2.2,1.1,1.7,['-3.2' '2.2'],['-3.2' '2.2' '1.1'],['-3.2' '2.2' '1.1' '1.7'],4,2020,2020Q2,...,1947.56,1754.64,1987.01,2119.01,110.41,97.41,99.07,116.45,102.29,118.39
74,2.2,1.1,1.7,0.8,['2.2' '1.1'],['2.2' '1.1' '1.7'],['2.2' '1.1' '1.7' '0.8'],7,2020,2020Q3,...,2249.37,2108.33,2029.60,1947.56,92.85,82.16,86.99,99.11,95.86,105.13
75,1.1,1.7,0.8,0.3,['1.1' '1.7'],['1.1' '1.7' '0.8'],['1.1' '1.7' '0.8' '0.3'],10,2020,2020Q4,...,2267.15,2327.89,2326.17,2249.37,113.80,94.52,100.72,107.69,98.61,107.70
76,1.7,0.8,0.3,1.2,['1.7' '0.8'],['1.7' '0.8' '0.3'],['1.7' '0.8' '0.3' '1.2'],1,2021,2021Q1,...,2976.21,2873.47,2591.34,2267.15,120.21,110.69,106.95,123.99,111.52,108.07


In [5]:
data["Y"]

0     2.5
1     1.8
2     2.0
3     1.1
4    -0.7
     ... 
72   -1.3
73   -3.2
74    2.2
75    1.1
76    1.7
Name: Y, Length: 77, dtype: float64

In [6]:
data = data[data.iloc[:, 9:].apply(lambda row : sum(pd.isna(row)), axis = 1) == 0]
data = data.reset_index(drop = True)
data["Y"]

0     2.5
1     1.8
2     2.0
3     1.1
4    -0.7
     ... 
72   -1.3
73   -3.2
74    2.2
75    1.1
76    1.7
Name: Y, Length: 77, dtype: float64

In [7]:
data['TIME'] = data['TIME'].astype(str)

time_list = []
for i in range(len(data)):
  time_i = data['TIME'][i]
  time = re.sub(r'(\d{4})(\d{2})', '\g<1>-\g<2>', time_i) #정규표현식 'YYYY-mm'
  time_list.append(time)
      
data['TIME'] = time_list
data['datetime'] = pd.to_datetime(data['TIME'],  format="%Y-%m") #datetime변환
data.sort_values('datetime', ignore_index = True, inplace=True)
data.set_index(['YEAR'], inplace=True)

In [8]:
train = data.drop(['Y','Y1','Y2','Y3', 'YY1','YY2','YY3','MONTH','QRT','TIME','STATE', 'datetime'], axis=1)
train.columns

Index(['X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2', 'X4_L3',
       'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1', 'X8_L2',
       'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2', 'X10_L3',
       'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2', 'X13_L3',
       'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1', 'X15_L2',
       'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1', 'X17_L2',
       'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'],
      dtype='object')

In [9]:
# train["X7_L2"] = train["X7_L2"] / 100000 # 10 ** 6
# train["X7_L3"] = train["X7_L3"] / 100000 # 10 ** 6

In [ ]:
# train["Y"] = data["Y"]

In [ ]:
# train["Y"]

In [ ]:
## 표준화
from sklearn.preprocessing import StandardScaler
# train = train.drop(['MONTH', 'YEAR', 'QRT', 'TIME', 'STATE', 'Y1',	'Y2',	'Y3',	'YY1',	'YY2',	'YY3', 'datetime'], axis=1)
# train = StandardScaler().fit_transform(train)
# pd.DataFrame(train)

In [ ]:
## 정규화
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
train

array([[0.10294118, 0.        , 0.58490566, ..., 0.        , 0.03074177,
        0.00414978],
       [0.77941176, 0.59210526, 0.67924528, ..., 0.021448  , 0.        ,
        0.        ],
       [1.        , 1.        , 0.8490566 , ..., 0.01977401, 0.06078937,
        0.03194364],
       ...,
       [0.10294118, 0.19736842, 0.        , ..., 0.71186441, 0.66283221,
        0.71125265],
       [0.25      , 0.23684211, 0.18867925, ..., 0.80163214, 0.69010313,
        0.73605482],
       [0.41176471, 0.36842105, 0.45283019, ..., 0.97216991, 0.81812773,
        0.73962555]])

In [ ]:
df = pd.DataFrame(train, columns=['X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3'], index=data.index)
df["Y"] = np.log1p(data["Y"])

In [ ]:
df

,X1_L2,X1_L3,X2_L2,X2_L3,X3_L2,X3_L3,X4_L2,X4_L3,X5_L2,X5_L3,...,X16_L1,X16_L2,X16_L3,X17_L1,X17_L2,X17_L3,X18_L1,X18_L2,X18_L3,Y
YEAR,,,,,,,,,,,,,,,,,,,,,
2002,0.102941,0.000000,0.584906,0.48,0.119093,0.000000,0.546875,0.608040,0.111301,0.000000,...,0.067586,0.033960,0.000000,0.000000,0.041160,0.010611,0.000000,0.030742,0.004150,1.252763
2002,0.779412,0.592105,0.679245,0.54,0.000000,0.012397,0.692708,0.653266,0.059932,0.028056,...,0.153942,0.121315,0.103645,0.037862,0.000000,0.000000,0.021448,0.000000,0.000000,1.029619
2002,1.000000,1.000000,0.849057,0.86,0.177694,0.090909,0.776042,0.844221,0.176370,0.114228,...,0.088554,0.109613,0.150115,0.026720,0.069951,0.040340,0.019774,0.060789,0.031944,1.098612
2002,0.676471,0.736842,0.754717,0.74,0.119093,0.090909,0.854167,0.783920,0.111301,0.080160,...,0.047361,0.079850,0.088818,0.049830,0.072384,0.048083,0.054719,0.065748,0.041498,0.741937
2003,0.455882,0.486842,0.849057,0.78,0.206049,0.138430,0.723958,0.763819,0.188356,0.146293,...,0.039290,0.074096,0.059700,0.065924,0.097222,0.087085,0.094057,0.098770,0.066300,-1.203973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020,0.220588,0.223684,0.603774,0.60,1.000000,1.000000,0.468750,0.562814,1.000000,0.997996,...,0.710921,0.750296,0.761920,0.852058,0.825629,0.802983,0.935865,0.824276,0.813163,NaN
2020,0.264706,0.302632,0.584906,0.68,0.809074,0.882231,0.328125,0.552764,0.613014,0.919840,...,0.521411,0.700220,0.779435,0.859899,0.732563,0.691139,0.893283,0.726597,0.839220,NaN
2020,0.102941,0.197368,0.000000,0.08,0.848771,0.838843,0.000000,0.351759,0.965753,0.861723,...,0.672705,0.721347,0.694920,0.678737,0.577960,0.575662,0.711864,0.662832,0.711253,1.163151


In [ ]:
df["Y"] = np.log1p(data["Y"])


In [ ]:
# X = train.drop(['Y'], axis=1)
# y = train['Y']

import os 

In [ ]:
def seed_everthing(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everthing(1)

In [ ]:
def rnn_model(X_train, y_train, X_test, y_test):
  model = Sequential()

  # model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))
  model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))

  # RNN 첫번째 은닉층 추가
  # model.add(LSTM(units=X_train.shape[1], activation='tanh', return_sequences = True))
  model.add(LSTM(units = 234, activation='tanh', return_sequences = True))
  # model.add(Dropout(0.6)) # 노드간의 커넥트를 랜덤하게 끊어줌
  model.add(Dropout(0.4))

  # RNN 두번째 은닉층 추가
  model.add(LSTM(int(234/2), activation='tanh', return_sequences = True))
  model.add(Dropout(0.4))

  # RNN 세번째 은닉층 추가
  model.add(LSTM(int(234/3), activation='tanh', return_sequences = True))
  model.add(Dropout(0.4))

  # RNN 네번째 은닉층 추가
  model.add(LSTM(10, activation='tanh', return_sequences = False))
  model.add(Dropout(0.4))

  # 출력층 추가
  model.add(Dense(units = 1))

  model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])
  checkpoint = ModelCheckpoint(filepath = f"./model_{i}.h5", monitor='val_loss', save_best_only=True)
  early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

  initial_learning_rate = 1e-4
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps = 100, decay_rate = 0.96, staircase=True)
  scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

  hist = model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=150,  batch_size = 8 ,callbacks=[checkpoint, early_stopping, scheduler])
  model_file = [fl for fl in os.listdir("/content/drive/MyDrive/hdf") if re.findall(f"model_{i}", fl)][0]
  model = load_model(f"/content/drive/MyDrive/hdf/{model_file}")

  return model

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from keras.backend import rnn
sample_size = len(df)
predicted_y = []

df_train = df[df.index < 2015] # 학습데이터
window = len(df_train) # 윈도우 사이즈 설정 (2002~2015)


df_test = df[df.index >= 2015] #검증데이터
forecast_horizon = len(df_test) # 검증데이터 길이

start_index = 0

# fit the model and make a forecast
for i in tqdm(range(forecast_horizon)): #검증데이터만큼 돌린다

    # writer = SummaryWriter()

    end_index = start_index + window 
    new_data = df.iloc[start_index : end_index] # df에서 end_index+1

    x_train, x_val, y_train, y_val = train_test_split(new_data.drop(["Y"], axis =1), new_data["Y"] ,test_size = 0.25, shuffle = False)

    # gets the trained neural network with given data
    model = rnn_model(x_train, y_train, x_val, y_val) #rnn_moel함수는 hist를 return해줌

    x_new = df.iloc[end_index]
    x_new = pd.DataFrame(x_new).T.drop(["Y"], axis = 1)
    y_pred = model.predict(x_new)
    # y_pred = y_pred[0][0]    

    predicted_y.append(y_pred)

    start_index = start_index + 1 # for rolling-window forecasts

    # window = window + 1 # for recursive forecasts
   

  0%|          | 0/25 [00:00<?, ?it/s]


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 10s 613ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 1s 199ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 1s 201ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 1s 201ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_err

  4%|▍         | 1/25 [01:14<29:43, 74.32s/it]


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 10s 621ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 1s 201ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 1s 200ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 1s 201ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_err

  8%|▊         | 2/25 [02:46<32:32, 84.90s/it]

...

 92%|█████████▏| 23/25 [32:22<02:44, 82.19s/it]


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 11s 648ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 1s 210ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 1s 205ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 1s 210ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_err

 96%|█████████▌| 24/25 [33:38<01:20, 80.26s/it]


Epoch 1: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 1/150
5/5 [==============================] - 10s 700ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 2: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 2/150
5/5 [==============================] - 1s 208ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 3: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 3/150
5/5 [==============================] - 1s 210ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_error: nan - lr: 1.0000e-04

Epoch 4: LearningRateScheduler setting learning rate to 9.999999747378752e-05.
Epoch 4/150
5/5 [==============================] - 1s 206ms/step - loss: nan - root_mean_squared_error: nan - val_loss: nan - val_root_mean_squared_err

100%|██████████| 25/25 [34:55<00:00, 83.80s/it]


In [ ]:
predicted_y

In [ ]:
predicted_y = [v[0][0] for v in predicted_y]

In [ ]:
df_test["loss"] = predicted_y - df_test["Y"]
df_test["predicted_y"] = predicted_y

In [ ]:
df_test["Y_exp"] = np.exp(df_test["Y"])
df_test["predicted_y_exp"] = np.exp(df_test["predicted_y"])
# df_test["predicted_y_r"] = round(df_test["predicted_y"], 1)

In [ ]:
df_test["Y"]

YEAR
2015    0.587787
2015    0.336472
2015    0.916291
2015    0.530628
2016    0.262364
2016    0.741937
2016    0.405465
2016    0.470004
2017    0.693147
2017    0.530628
2017    0.916291
2017   -0.356675
2018    0.741937
2018    0.470004
2018    0.530628
2018    0.587787
2019   -0.223144
2019    0.693147
2019    0.336472
2019    0.832909
2020         NaN
2020         NaN
2020    1.163151
2020    0.741937
2021    0.993252
Name: Y, dtype: float64

In [ ]:
df_test[["Y_exp", "predicted_y_exp", "loss"]]#.to_csv("./prediction_result.csv", sep = ",", encoding = "cp949")

,Y_exp,predicted_y_exp,loss
YEAR,,,
2015,1.8,1.883491,0.045340
2015,1.4,1.895097,0.302798
2015,2.5,1.890210,-0.279603
2015,1.7,1.905734,0.114239
2016,1.3,1.911419,0.385481
2016,2.1,1.904399,-0.097771
2016,1.5,1.907272,0.240209
2016,1.6,1.893813,0.168589
2017,2.0,1.886347,-0.058505


In [ ]:
print((np.sum((df_test['predicted_y_exp'] - df_test["Y_exp"]) ** 2) / len(df_test['predicted_y_exp'])) ** 1/2)

0.16062710583309672


In [ ]:
 y_vloss = hist.history['loss']
y_loss = hist.history['val_loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()

In [ ]:
np.sqr(predicted_y_list)

In [ ]:
## 교차검증 시 데이터 분할 시각화 helper function 가져오기
from sklearn.model_selection import TimeSeriesSplit 
n_split = 4
tscv = TimeSeriesSplit(n_splits=n_split)
X = X.set_index(data["datetime"])

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
  print("Fold: {}".format(fold))
  print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
  print("\n")
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

  

### covid 미포함

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gdp_re/training_data_0524.csv')
data = data.drop(['Unnamed: 0'], axis=1)
data.tail()

In [ ]:
data = data[data.iloc[:, 9:].apply(lambda row : sum(pd.isna(row)), axis = 1) == 0]
data = data.reset_index(drop = True)
data

In [ ]:
data = data[(data['YEAR']!=2020)&(data['YEAR']!=2021)]
data

In [ ]:
data['TIME'] = data['TIME'].astype(str)

time_list = []
for i in range(len(data)):
  time_i = data['TIME'][i]
  time = re.sub(r'(\d{4})(\d{2})', '\g<1>-\g<2>', time_i) #정규표현식 'YYYY-mm'
  time_list.append(time)
      
data['TIME'] = time_list
data['datetime'] = pd.to_datetime(data['TIME'],  format="%Y-%m") #datetime변환
data.sort_values('datetime', ignore_index = True, inplace=True)

In [ ]:
data.tail()

In [ ]:
train = data.drop(['Y1','Y2','Y3','YY1','YY2','YY3','MONTH','YEAR','QRT','TIME','STATE','datetime'], axis=1)
train.columns
## 다 떄려넣기
## 성능확인
## 너무 다른얟,림ㅇ

In [ ]:
# ##정규화
# data['ride_passen_log'] = np.log1p(data['ride_passen'])
# data['takeoff_passen_log'] = np.log1p(data['takeoff_passen'])
# data.head(13)

## 표준화
from sklearn.preprocessing import StandardScaler
# train = train.drop(['MONTH', 'YEAR', 'QRT', 'TIME', 'STATE', 'Y1',	'Y2',	'Y3',	'YY1',	'YY2',	'YY3', 'datetime'], axis=1)
train = StandardScaler().fit_transform(train)
train

In [ ]:
# ## 정규화
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
train

In [ ]:
train = pd.DataFrame(train, columns=[['Y', 'X1_L2', 'X1_L3', 'X2_L2', 'X2_L3', 'X3_L2', 'X3_L3', 'X4_L2',
       'X4_L3', 'X5_L2', 'X5_L3', 'X6_L2', 'X6_L3', 'X7_L2', 'X7_L3', 'X8_L1',
       'X8_L2', 'X8_L3', 'X9_L1', 'X9_L2', 'X9_L3', 'X10_L1', 'X10_L2',
       'X10_L3', 'X11_L1', 'X11_L2', 'X11_L3', 'X12_L2', 'X12_L3', 'X13_L2',
       'X13_L3', 'X14_L0', 'X14_L1', 'X14_L2', 'X14_L3', 'X15_L0', 'X15_L1',
       'X15_L2', 'X15_L3', 'X16_L0', 'X16_L1', 'X16_L2', 'X16_L3', 'X17_L1',
       'X17_L2', 'X17_L3', 'X18_L1', 'X18_L2', 'X18_L3']])
train

In [ ]:
X = train.drop(['Y'], axis=1)
y = train['Y']

In [ ]:
## 교차검증 시 데이터 분할 시각화 helper function 가져오기
from sklearn.model_selection import TimeSeriesSplit 
n_split = 4
tscv = TimeSeriesSplit(n_splits=n_split)
X = X.set_index(data["datetime"])

for fold, (train_index, test_index) in enumerate(tscv.split(X)):
  print("Fold: {}".format(fold))
  print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
  print("\n")
  X_train, X_test = X.iloc[train_index], X.iloc[test_index]
  y_train, y_test = y.iloc[train_index], y.iloc[test_index]

In [ ]:
def seed_everthing(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  tf.random.set_seed(seed)

seed_everthing(1)

In [ ]:
X_train

In [ ]:
model = Sequential()

# model.add(keras.Input(shape=(X_train.shape[1],1)))
# model.add(Dense(self.units, activation='tanh')
# model.layers.Embedding(67, 48, input_length = 1)
# model.add(layers.Embedding(input_dim = 67, output_dim = 1))
model.add(Embedding(input_dim =X_train.shape[1], output_dim = 1))

# RNN 첫번째 은닉층 추가
model.add(SimpleRNN(units=X_train.shape[1], activation='tanh', return_sequences = True))
model.add(Dropout(0.6)) # 노드간의 커넥트를 랜덤하게 끊어줌

# RNN 두번째 은닉층 추가
model.add(SimpleRNN(int(X_train.shape[1]/2), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.5))

# RNN 세번째 은닉층 추가
model.add(SimpleRNN(int(X_train.shape[1]/3), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.3))

# # RNN 네번째 은닉층 추가
# model.add(SimpleRNN(int(self.units/4), activation='tanh', return_sequences = True)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
# model.add(Dropout(0.3))

model.add(SimpleRNN(10, activation='tanh', return_sequences = False)) # True : many to many, 혹은 여러 RNN / LSTM 층을 쌓을때 사용, False = many to one, Dense 층으로 넘겨줄때
model.add(Dropout(0.3))

# # RNN 세번째 은닉층 추가
# # model.add(SimpleRNN(self.units, activation='tanh', return_sequences=True))
# model.add(Dense(units = self.units / 4, activation="ReLU"))
# # model.add(SimpleRNN(units = 10, activation='tanh', return_sequences=True))
# model.add(Dropout(0.2))

# 출력층 추가: 분류모델 제외하고 활성화함수 사용X
model.add(Dense(units = 1)) # 다대일

## [3] compile: RNN모델을 학습시키기 위한 학습과정 설정하는 단계
model.compile(optimizer='adam', loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()])

## [4] 학습데이터로 RNN모델 학습
# earlystopping 선언
save_folder = '/content/drive/MyDrive/Colab Notebooks/교수님/hdf5/'
model_path = save_folder + 'mnist-' + '{epoch:02d}-{val_loss:.4f}.hdf5'

checkpoint = ModelCheckpoint(filepath=model_path, monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience =60, mode='min')

initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate, decay_steps = 100, decay_rate = 0.96, staircase=True)
scheduler = keras.callbacks.LearningRateScheduler(lr_schedule, verbose=1)

hist = model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=100,  batch_size =16 ,callbacks=[checkpoint, early_stopping, scheduler])

In [ ]:
y_vloss = hist.history['loss']
y_loss = hist.history['val_loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
plt.show()